Please note that this tutorial is focused at reconstructing CSD
at a subset of the _CSD_ grid.  For sake of simplicity it uses kCSD
(cross)kernels only.
To learn, how to create kESI (cross)kernels please consult
`tutorial_*_basics_explained.ipynb`.  To compare the reconstructed CSD
with kCSD reconstruction at all nodes of the _CSD_ grid please run
one of `tutorial_slice[_basics_explained].ipynb` notebooks.

# Requirements

## Memory

The code of the notebook requires at least 1.8 GB (1.7 GiB) of free memory.


## Environment

1. Anaconda Python distribution (tested with `Miniconda3-py39_4.12.0-Linux-x86_64.sh`, _conda v. 4.12.0_).
2. Jupyter server (see `extras/jupyter_server.sh` for details).
3. Anaconda environments (run `setup_conda_envs.sh`).

# Kernel construction tools

## Electrode object

The implementation of the electrode object is minimal necessary for construction of a kCSD (cross)kernel.

In [ ]:
import collections

Electrode = collections.namedtuple('Electrode',
                                   ['x', 'y', 'z', 'conductivity'])

We use the same positions of electrodes as `tutorial_slice[_basics_explained].ipynb` notebooks.

In [ ]:
CONDUCTIVITY = 0.3  # S/m

ELECTRODES_XYZ = [(0.0, 0.0, 5e-05),
                  (5e-05, 0.0, 0.00015),
                  (5e-05, -5e-05, 0.00025)]

electrodes = [Electrode(x, y, z, CONDUCTIVITY) for x, y, z in ELECTRODES_XYZ]

## Model source

We want to use CSD bases 36μm wide ($R = 18\mu{}m$).

In [ ]:
from kesi.common import SphericalSplineSourceKCSD

SRC_R = 18e-6  # m

spline_nodes = [SRC_R / 3, SRC_R]
spline_polynomials = [[1],
                      [0,
                       6.75 / SRC_R,
                       -13.5 / SRC_R ** 2,
                       6.75 / SRC_R ** 3]]
model_src = SphericalSplineSourceKCSD(0, 0, 0,
                                      spline_nodes,
                                      spline_polynomials)

## Convolver object

In [ ]:
import numpy as np
from kesi.kernel.constructor import Convolver

ROMBERG_K = 5
Z_MIN = 0
Z_MAX = 3e-4
XY_AMP = 1.5e-4

_h_min = SRC_R * 2**(1 - ROMBERG_K)
_X = _Y = np.linspace(-XY_AMP, XY_AMP, int(np.floor(2 * XY_AMP / _h_min)) + 1)
_Z = np.linspace(Z_MIN, Z_MAX, int(np.floor((Z_MAX - Z_MIN) / _h_min)) + 1)

_csd_grid = _pot_grid = [_X, _Y, _Z]

convolver = Convolver(_pot_grid, _csd_grid)

for _c, _h in zip("XYZ", convolver.steps("POT")):
    assert _h >= _h_min, f"{_c}:\t{_h} < {_h_min}"
    if _h >= 2 * _h_min:
        print(f"You can reduce number of nodes of quadrature for {_c} dimension")

## Convolver interface

In [ ]:
from kesi.kernel.constructor import ConvolverInterfaceIndexed
from scipy.integrate import romb

ROMBERG_N = 2 ** ROMBERG_K + 1
ROMBERG_WEIGHTS = romb(np.identity(ROMBERG_N), dx=2 ** -ROMBERG_K)

SRC_MASK = ((convolver.SRC_Z > Z_MIN + SRC_R)
            & (convolver.SRC_Z < Z_MAX - SRC_R)
            & (abs(convolver.SRC_X) < XY_AMP - SRC_R)
            & (abs(convolver.SRC_Y) < XY_AMP - SRC_R))

In [ ]:
print(SRC_MASK.sum())

In [ ]:
convolver_interface = ConvolverInterfaceIndexed(convolver,
                                                model_src.csd,
                                                ROMBERG_WEIGHTS,
                                                SRC_MASK)

## Potential Basis Functions

In [ ]:
from kesi.kernel.potential_basis_functions import Analytical as PBF

In [ ]:
pbf = PBF(convolver_interface,
          potential=model_src.potential)

## Kernel constructor and cross-kernel constructor

In [ ]:
from kesi.kernel.constructor import KernelConstructor, CrossKernelConstructor

kernel_constructor = KernelConstructor()

### Cross-kernel for reconstruction in coordinate planes

To calculate the cross-kernel matrix we need to select nodes of the _CSD_ grid.
We are going to visualise current source density in the coordinate planes,
thus in the boolean mask we select the closest nodes to the planes.
First we unequivocally define the planes by their intersection point.

In [ ]:
coordinate_x = 25e-6
coordinate_y = -25e-6
coordinate_z = 150e-6

intersection = [coordinate_x,
                coordinate_y,
                coordinate_z]

We find indices of the node of the _CSD_ grid closest to the intersection in terms of Manhattan distance.

In [ ]:
indices_of_coordinates = [np.argmin(abs(_C - _c))
                          for _C, _c in zip(convolver.CSD_GRID,
                                            intersection)]

With the indices we select the closest (to the coordinate planes) nodes of the _CSD_ grid.
<!-- We define an auxilary function `one_hot(i, n)` which returns `n`-long vector which all elements but `i`-th are `0` (and the `i`-th element is `1`). -->

In [ ]:
CSD_MASK = np.zeros(convolver.csd_shape,
                    dtype=bool)
CSD_MASK[indices_of_coordinates[0], :, :] = True  # X-coordinate plane
CSD_MASK[:, indices_of_coordinates[1], :] = True  # Y-coordinate plane
CSD_MASK[:, :, indices_of_coordinates[2]] = True  # Z-coordinate plane

We count the selected nodes.

In [ ]:
n_csd_nodes = CSD_MASK.sum()
print(f'{n_csd_nodes} nodes of the CSD grid selected.')

We use the `CSD_MASK` to create a cross-kernel constructor.

In [ ]:
kernel_constructor.crosskernel = CrossKernelConstructor(convolver_interface,
                                                        CSD_MASK)

To retrieve three CSD planes from CSD vector we define an auxilary function `to_planes()`.
The function uses three index arrays to select (and arrange) appropriate elements of the vector.

In [ ]:
# As we reconstruct CSD at n_csd_nodes points,
# n_csd_nodes is invalid index value for the
# reconstructed CSD vector.

_CSD_IDX = np.full_like(CSD_MASK, n_csd_nodes,
                        dtype=np.int32)
_CSD_IDX[CSD_MASK] = np.arange(n_csd_nodes)

COORDINATE_PLANE_INDICES = [_CSD_IDX[indices_of_coordinates[0], :, :].copy(),
                            _CSD_IDX[:, indices_of_coordinates[1], :].copy(),
                            _CSD_IDX[:, :, indices_of_coordinates[2]].copy()
                            ]
del _CSD_IDX

# We test, whether all indices are valid.

for _A in COORDINATE_PLANE_INDICES:
    assert _A.min() >= 0 and _A.max() < CSD_MASK.sum()
    
def to_planes(CSD):
    return [CSD[IDX] for IDX in COORDINATE_PLANE_INDICES]

# Reconstructor

## Construction of kernels

In [ ]:
%%time
B = kernel_constructor.potential_basis_functions_at_electrodes(electrodes,
                                                               pbf)

In [ ]:
KERNEL = kernel_constructor.kernel(B)

In [ ]:
%%time
CROSSKERNEL = kernel_constructor.crosskernel(B)

In [ ]:
del B  # the array is large and no longer needed

## Reconstructors

In [ ]:
from kesi._verbose import _CrossKernelReconstructor as Reconstructor
from kesi._engine import _LinearKernelSolver as KernelSolver


reconstructor = Reconstructor(KernelSolver(KERNEL),
                              CROSSKERNEL)

# Visualisation

In [ ]:
from tutorial import coordinate_planes_CoordinatePlanesVisualisation as CoordinatePlanesVisualisation

In [ ]:
csd_plotter = CoordinatePlanesVisualisation([_x.flatten() for _x in convolver.CSD_GRID],
                                            intersection,
                                            unit_factor=1e-12,
                                            unit='$\\frac{\\mu{}A}{mm^3}$',
                                            length_factor=1e6,
                                            length_unit='$\\mu{}m$')

# Reconstruction

otential values (given in $\mu{}V$) are stored in a vector `POTENTIAL`.
Each of its $N$ elements were calculated from ground truth CSD
in the `tutorial_slice.ipynb` notebook (`GT_V` therein).

In [ ]:
POTENTIAL = [-126548.99283768,
             -119140.53772061,
             -73225.23872045,
             ]

As potential input was a vector, the reconstructor returns
a vector of CSD values.  Each of its $\tilde{\underline{N}}$
elements corresponds to a selected node of the _CSD_ grid.

In [ ]:
%%time
CSD = reconstructor(POTENTIAL)

It should be same as the kCSD reconstruction in
`tutorial_slice[_basics_explained].ipynb` notebooks.

In [ ]:
csd_plotter.plot_planes(to_planes(CSD),
                        title='kCSD reconstruction from slice tutorial notebook')